# 1 Introduction & Objectif

## Benchmarking ResNet50 Model for Image Classification

### 📌 Objective:
This notebook replicates the benchmark model from Rakuten’s challenge.
We implement a **ResNet50** model with:
- **Pretrained ImageNet weights**
- **27 layers unfrozen (8 convolutional layers)**
- **Image preprocessing using ImageDataGenerator**
- **Evaluation using Weighted F1-score**  

### 🎯 Expected Outcome:
- Establish a **baseline** using the benchmark model.
- Compare the **F1-score with 0.5534 (Benchmark Score)**.
- Prepare for **further improvements** (Data Augmentation, Fine-tuning).


# 2. Imports & Configuration

In [ ]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense
from keras.models import Model
from keras.callbacks import Callback
from sklearn.metrics import f1_score, precision_score, recall_score


# 3. Définition du modèle ResNet50

## Modèle utilisé dans le benchmark Rakuten
Le modèle d’image utilisé dans le benchmark est **ResNet50 pré-entraîné sur ImageNet**.  
**Modifications apportées :**
- **27 couches décongelées (dont 8 convolutionnelles)**.
- **Ajout d’une couche Dense(512, activation='relu')**. ????
- **Classification single-label avec Softmax**.


**Nombre de paramètres :**
- **12,144,667 paramètres entraînables**.
- **23,643,035 paramètres non entraînables**.

https://challengedata.ens.fr/participants/challenges/35/

*For the image data, a version of Residual Networks (ResNet) model (reference) is used. ResNet50 implementation from Keras is used as the base model. The details of the basic benchmark model can be found in this notebook. The model is a pre-trained ResNet50 with ImageNet dataset. 27 different layers from top are unfrozen, which include 8 Convolutional layers for the training. The final network contains 12,144,667 trainable and 23,643,035 non-trainable parameters.*

In [ ]:
def create_resnet_model(input_shape=(224, 224, 3), num_classes=27):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape, pooling='avg')

    # : Décongeler 27 couches
    for layer in base_model.layers[-27:]:
        layer.trainable = True  

    x = Dense(512, activation='relu')(base_model.output)
    output = Dense(num_classes, activation='softmax')(x)  # Softmax pour single-label classification

    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


# 4. Chargement des images avec ImageDataGenerator

We use `ImageDataGenerator` for:
- **Rescaling** pixel values (1/255).
- **Loading images from directories** (`train` & `test`).
- **Setting image size to (224, 224)** (like ResNet50 expects).
- **Data split : 80% train / 20% validation** .

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    'data/raw_images/image_train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',  
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    'data/raw_images/image_train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    subset='validation'
)


# 5. Model Training 

- **Batch size** : 32
- **Nombre d’epochs** : 10
- **Évaluation** : Sur validation set.

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator)
)


# 6. Model Evaluation

## 6.1 Learning Curves: Accuracy & Loss

In [ ]:
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title("Learning Curve - ResNet50 Benchmark Model")
plt.show()


## 6.2  Evaluation Metrics: Weighted F1-Score, Precision & Recall

- **F1-score weighted**
- **Précision**
- **Recal**

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Obtenir les vraies étiquettes
y_true = val_generator.classes

# Obtenir les prédictions du modèle
y_pred = np.argmax(model.predict(val_generator), axis=1)

# Calcul des métriques principales
f1 = f1_score(y_true, y_pred, average="weighted")
precision = precision_score(y_true, y_pred, average="weighted")
recall = recall_score(y_true, y_pred, average="weighted")

print(f"🔹 Weighted F1-score: {f1:.4f}")
print(f"🔹 Precision: {precision:.4f}")
print(f"🔹 Recall: {recall:.4f}")


## 6.3 Detailed Classification Report


In [ ]:
# Générer un rapport détaillé de classification
report = classification_report(y_true, y_pred, target_names=val_generator.class_indices.keys())

print("🔹 Classification Report:\n")
print(report)


## 6.4 Confusion Matrix: Error Analysis

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calculer la matrice de confusion
cm = confusion_matrix(y_true, y_pred)

# Affichage sous forme de heatmap
plt.figure(figsize=(16, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=val_generator.class_indices.keys(), yticklabels=val_generator.class_indices.keys())
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix - ResNet50 Benchmark")
plt.show()


# 8. Benchmark Results & Next Steps

 ### 🔹 Results:
- **Benchmark F1-score (given)**: `0.5534`
- **Our Model F1-score**: `{f1:.4f}`  
- **Precision**: `{precision:.4f}`
- **Recall**: `{recall:.4f}`  

### 🔹 Next Steps:
✅ Compare **baseline** with **Data Augmentation** (`02_Data_Augmentation`).  
✅ Perform **Fine-Tuning** on top layers (`03_Augmentation_FineTuning`).  
✅ Optimize **Learning Rate (`04_LR_Optimization`)**.  
✅ Integrate **Multimodal (Text + Image) classification** (`05_Bimodal_Integration`)..  
# 📌 Benchmark Results & Next Steps
### 🔹 Results:
- **Benchmark F1-score (given)**: `0.5534`
- **Our Model F1-score**: `{f1:.4f}`  
- **Precision**: `{precision:.4f}`
- **Recall**: `{recall:.4f}`  

### 🔹 Next Steps:
✅ Compare **baseline** with **Data Augmentation** (`02_Data_Augmentation`).  
✅ Perform **Fine-Tuning** on top layers (`03_Augmentation_FineTuning`).  
✅ Optimize **Learning Rate (`04_LR_Optimization`)**.  
✅ Integrate **Multimodal (Text + Image) classification** (`05_Bimodal_Integration`).  

